In [46]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

Sistema de recomendación item-item:

    def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Ruta relativa para traer el dataset limpio que vamos a utilizar y creamos un dataframe para trabajar

In [47]:
# creamos las variables para las Rutas relativas a los archivos JSON
ruta_Output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')
# Creamos el dataframe
games = pd.read_parquet(ruta_Output_Steam_Games)
# Solo nos quedamos con las columnas que vamos a necesitar
modelo_df = games.loc[:, ["Genres", "ItemId",'ItemName']]

In [48]:
# Asignamos la columna 'Genres' a una variable para luego tratarla y limpiar caracteres
genres_variable = modelo_df['Genres'].copy()
# Convertimos a cadena antes de reemplazar los corchetes que queremos sacar
genres_variable = genres_variable.apply(lambda x: str(x).replace('[', '').replace(']', ''))
# Eliminamos las comillas y agregamos comas entre los géneros para separalos
genres_variable = genres_variable.apply(lambda x: ', '.join(filter(None, map(str.strip, x.split("'")))) if isinstance(x, str) else x)
# Reemplazamos la columna 'Genres' en el DataFrame original con la nueva columna tratada
modelo_df['Genres'] = genres_variable
# Imprimimos para ver como quedo la columna después de reemplazar
print(modelo_df['Genres'])

0              Action, Casual, Indie, Simulation, Strategy
1                       Free to Play, Indie, RPG, Strategy
2          Casual, Free to Play, Indie, Simulation, Sports
3                                Action, Adventure, Casual
4                            Action, Indie, Casual, Sports
                               ...                        
31989                  Casual, Indie, Simulation, Strategy
31990                              Casual, Indie, Strategy
31991                            Indie, Racing, Simulation
31992                                        Casual, Indie
31993    Early Access, Adventure, Indie, Action, Simula...
Name: Genres, Length: 31994, dtype: object


In [49]:
#vamos a cambiar unos errores que hay en la columna Genres
# como vemos hay unos errores de caracteres, que vamos a solucionar de la siguiente manera
modelo_df['Genres'] = modelo_df['Genres'].replace('Design &amp; Illustration', 'Design & Illustration')
modelo_df['Genres'] = modelo_df['Genres'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [50]:
print(modelo_df['Genres'].unique())

['Action, Casual, Indie, Simulation, Strategy'
 'Free to Play, Indie, RPG, Strategy'
 'Casual, Free to Play, Indie, Simulation, Sports' ...
 'Action, Massively Multiplayer, Strategy'
 'Video Production, Utilities, Web Publishing'
 'Early Access, Adventure, Indie, Action, Simulation']


Vamos a empezar con el modelo de machine learning

CountVectorizer es una clase de la biblioteca scikit-learn que se utiliza para convertir una colección de documentos de texto en una matriz de recuento de términos o tokens. 

Toma un conjunto de documentos de texto y crea una representación numérica de esos documentos en forma de una matriz donde cada fila representa un documento y cada columna representa una palabra única en el conjunto de documentos. La entrada en cada celda de la matriz indica cuántas veces aparece esa palabra en el documento correspondiente.

In [51]:
# Cargamos en la variable cv, CountVectorizer que es la herramienta de preprocesamiento que convierte datos 
# de texto en una representación numérica adecuada para ser utilizada por algoritmos de aprendizaje automático
cv = CountVectorizer()

In [52]:
# ahora vamos a ajustar la herramienta CountVectorizer al contenido de la columna 'Genres' 
# y transformar esos datos en una matriz de recuento de términos
# Con .toarray(): Convierte la representación dispersa de la matriz de recuento de términos a una matriz densa.
# .shape: Devuelve la forma (número de filas y columnas) de la matriz resultante.
cv.fit_transform(modelo_df['Genres']).toarray().shape

(31994, 33)

In [53]:
# Se generan los vectores a comparar 
vectores = cv.fit_transform(modelo_df['Genres']).toarray()

In [54]:
# Mostramos como quedo Nuestra variable vectores
vectores

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

cosine_similarity() nos va a servir para Calcular la similitud coseno entre todos los pares de vectores en la matriz.

La similitud coseno es una medida de similitud entre dos vectores en un espacio multidimensional que mide el coseno del ángulo entre ellos. Se utiliza comúnmente para comparar la similitud entre documentos en el análisis de texto.

In [55]:
# Aplicamos la similitud del coseno a nuestros vectores
similitud = cosine_similarity(vectores)

In [56]:
# Se obtiene el array de similitud
similitud[0]

array([1.        , 0.36514837, 0.50709255, ..., 0.51639778, 0.63245553,
       0.54772256])

In [57]:
# Ordenamos la similitud entre más similar a menos similar tomando 5 valores
sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x:x[1])[1:6]

[(3336, 0.9999999999999999),
 (3553, 0.9999999999999999),
 (5340, 0.9999999999999999),
 (5690, 0.9999999999999999),
 (6903, 0.9999999999999999)]

Ahora que tenemos la matriz ya ordenada y tomamos los valores que necesitamos, vamos a generar una funcion para poder despues crear una columna y guardar los datos para agilizar la funcion consulta

In [59]:
# Se genera una función que te da la recomendación por título
def recomendacion(id_juego):
    # Obtenemos el índice del juego con el ID dado
    indice_juego = modelo_df[modelo_df["ItemId"] == id_juego].index[0]

    # Obtenemos las distancias de similitud para el juego dado
    distances = similitud[indice_juego]

    # Ordenamos las distancias en orden descendente y obtener las 5 recomendaciones principales (excluyendo el juego dado)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Obtenemos los nombres de los juegos recomendados
    recommended_titles = [modelo_df.iloc[i[0]]['ItemName'] for i in lista_juegos]

    return recommended_titles

Vamos a aplicar la funcion al dataframe modelo_df para crear la columna RecomendacionesTop5 con todas las recomendaciones que nos arrojo el array y la funcion

In [60]:
# Se aplica la función al dataframe para obtener una nueva columna con las recomendaciones ya que es más facil 
# de cargar y leer para la funcion
modelo_df['RecomendacionesTop5'] = modelo_df['ItemId'].apply(recomendacion)

In [61]:
# Vemos como nos quedaron las columnas y el peso del dataframe
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Genres               31994 non-null  object
 1   ItemId               31994 non-null  int32 
 2   ItemName             31994 non-null  object
 3   RecomendacionesTop5  31994 non-null  object
dtypes: int32(1), object(3)
memory usage: 875.0+ KB


In [62]:
# Una vez echa la columna de recomendaciones podemos borrar los demas campos que no precisamos
modelo_df.drop(columns=['ItemName','Genres'], inplace=True)

In [63]:
# Chequeamos
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ItemId               31994 non-null  int32 
 1   RecomendacionesTop5  31994 non-null  object
dtypes: int32(1), object(1)
memory usage: 375.1+ KB


Por ultimo creamos el CSV para alimentar la funcion consulta

In [64]:
# Guardamos el resultado en un nuevo archivo CSV
modelo_df.to_csv('consulta6.csv', index=False)